In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Data Preparation

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:08<00:00, 3269011.06it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 282348.57it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5442413.00it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8755992.29it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [14]:
for X, y in test_dataloader:
    print(f"Shape of X [N (batch size), C (channel), H (height), W (width)]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N (batch size), C (channel), H (height), W (width)]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Training

1. Create a class that inherits from nn.Module
2. Define layers of network in __init__
3. Specify how data will pass through the network in forward function
4. Move operations to GPU or MPS if available

In [15]:
device=("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using mps device


In [16]:
# define training model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # initializer of superclass
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # forward and calculate prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # backprop
        # loss.backward() omputes the gradient of the loss with 
        # respect to the model parameters (i.e., weights and biases) using backpropagation
        loss.backward()
        # optimizer.step() updates the model parameters using the gradients computed in the previous step
        optimizer.step()
        # optimizer.zero_grad() resets the gradient to 0 for the next pass
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): # disable gradient calculation for memory efficiency
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # .item() converts the loss tensor to a Python scalar and adds it to test_loss
            test_loss += loss_fn(pred, y).item()
            # pred.argmax(1) computes the index of the maximum value along dimension 1
            # (pred.argmax(1) == y) creates a boolean tensor indicating which predictions are correct
            # .type(torch.float) converts the boolean tensor to a float tensor (where True becomes 1.0 and False becomes 0.0)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294185 [   64/60000]
loss: 2.287882 [ 6464/60000]
loss: 2.266909 [12864/60000]
loss: 2.265497 [19264/60000]
loss: 2.250823 [25664/60000]
loss: 2.199007 [32064/60000]
loss: 2.222300 [38464/60000]
loss: 2.176679 [44864/60000]
loss: 2.181256 [51264/60000]
loss: 2.141449 [57664/60000]
Test Error: 
 Accuracy: 34.8%, Avg loss: 2.141877 

Epoch 2
-------------------------------
loss: 2.150320 [   64/60000]
loss: 2.150537 [ 6464/60000]
loss: 2.089576 [12864/60000]
loss: 2.103669 [19264/60000]
loss: 2.065547 [25664/60000]
loss: 1.989791 [32064/60000]
loss: 2.025140 [38464/60000]
loss: 1.942077 [44864/60000]
loss: 1.949384 [51264/60000]
loss: 1.873759 [57664/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.874818 

Epoch 3
-------------------------------
loss: 1.907107 [   64/60000]
loss: 1.886307 [ 6464/60000]
loss: 1.763430 [12864/60000]
loss: 1.798098 [19264/60000]
loss: 1.708212 [25664/60000]
loss: 1.652727 [32064/60000]
loss: 1.674282 [38464/

__call__ Method: When calling model(X), the __call__ method of the nn.Module class is invoked.

Preprocessing: The __call__ method handles any necessary preprocessing, such as hooks and other internal mechanisms.

Forward Pass: The __call__ method then calls the forward method, passing the input X to it.

Postprocessing: After the forward method completes, the __call__ method handles any necessary postprocessing.
